## Notebook Contains

A Rag Agent using `Cewai`

## Importing Libraries

In [1]:
# Importing Necessaet Libraries
import os
import sys
import io
from crewai import LLM, Agent, Task, Crew
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_chroma import Chroma
# from langchain.text_splitter import RecursiveCharacterTextSplitter

from crewai.tools import BaseTool

In [2]:
# --- Configuration --- #
MODEL_NAME = "ollama/openhermes:latest"


## Definition of Tools

In [3]:
class CodingAgent(BaseTool):
    name: str = "CodingAgent"
    description: str = """A coding agent that can write and execute Python code based on user queries.
    It returns the output of the code if successful, or an error message if an error occurs.
    The query must be a valid Python code block.
    """

    def _run(self, query: str) -> str:
        """Run the coding agent to write and execute Python code based on the query."""
        
        # We need to capture the output, so we redirect stdout to a string buffer.
        old_stdout = sys.stdout
        redirected_output = io.StringIO()
        sys.stdout = redirected_output
        
        try:
            # Execute the user's query as a code block.
            # The 'exec' function can run multi-line statements, including imports and function definitions.
            exec(query)
            
            # Get the output from the redirected buffer.
            output = redirected_output.getvalue()
            
            # Restore the original stdout.
            sys.stdout = old_stdout
            
            # If there's no output, we return a message indicating success.
            if not output.strip():
                return "Code executed successfully with no output."
            
            # Return the captured output.
            return output
            
        except Exception as e:
            # If any error occurs during execution, capture it and return the error message.
            # Restore the original stdout first.
            sys.stdout = old_stdout
            return f"An error occurred while executing the code: {e}"

In [4]:
coading_tool = CodingAgent()

print(coading_tool._run("print('Hello')"))  # Test the tool

Hello



## Initializing LLM

In [5]:
# --- Configuring the LLM --- #
llm = LLM(
    model= MODEL_NAME,
    temperature=0.7
)


In [6]:
# Instantiate the tools
retrival_tool = CodingAgent()

# define Agent first
agent = Agent(
    role="Coading Agent",
    goal="""Write python code based on user queries and answer executing code.""",
    backstory= """You are a helpful assistant. You can write and execute Python code based on user queries and return the output,
      executing the code.""",
    # verbose=True,
    verbose=False,
    tools=[retrival_tool],
    llm=llm
)

# Define Task
task = Task(
    description=f"""Write clean python code based on the user query.
    User Query: {{query}}""",
    expected_output="return the code like ```CODE:\n``` and the output of the code.",
    agent=agent
)

# Define Crew
crew = Crew(
    agents=[agent],
    tasks=[task],
    # verbose=True
    verbose=False
)

In [7]:
# Checking if Crew is ready
print(crew.kickoff(inputs={"query": "write a python code to print 'Hello World'."}) )

CODE: `print("Hello World")`
Output: `Hello World`


In [8]:
# Defning Main Function
def main():
    """Main function to kickoff the Crew."""
    print("\n\t--- Starting Crew for RAG Agent ---")
    
    while True:
        user_query = input("Enter your query (or type 'exit' to quit): ")
        print(f"\n🙎‍♂️: {user_query}")
        if user_query.lower() == 'exit':
            print("Exiting the Crew. Goodbye!")
            break
        
        # Kickoff the Crew with the user query
        response = crew.kickoff(inputs={"query": user_query})
        print(f"🤖: {response}")

In [10]:
main()


	--- Starting Crew for RAG Agent ---

🙎‍♂️: write a python code to generate a tringle with star. The Trangle will be horizontally symmetric. user can give number of rows. show output for n = 5
🤖: CODE: n = 5
for i in range(n):
    print('*' * (2 * i + 1))

OUTPUT:
*
***
*****
*******
*********

🙎‍♂️: Now create a triagle with natural numbers. Code should be modular. User will give the value for n. show output for n=4
🤖: ```
CODE:
def create_triangle(n):
    for i in range(1, n+1):
        for j in range(i):
            print(j + 1, end=" ")
        print()

# Example usage: create_triangle(4)
create_triangle(4)

Output: 
1 
1 2 
1 2 3 
1 2 3 4 
```

🙎‍♂️: Write a pyython function to create a natural number triangle. the format of the triangle will be a center. and the user will give the row number. Show the result for n = 5
🤖: CODE:
```python
def triangle(n):
    for i in range(1, n+1):
        row = ''
        for j in range(i):
            row += str(j + 1) + ' '
        print(row.r